# Clustering Crypto

In [159]:
# Initial imports
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [95]:
# Load the crypto_data.csv dataset.
from operator import index


file = Path("crypto_data.csv")
crypto_df = pd.read_csv(file, index_col=0)
crypto_raw = crypto_df.copy()
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [96]:
crypto_df.dtypes


CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [97]:
# Identify unique values in the IsTrading column
np.unique(crypto_df.IsTrading)

array([False,  True])

In [98]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df.apply(lambda row: row['IsTrading'] == True, axis=1)]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [99]:
# Verify that only True Values remain in the IsTraining column
np.unique(crypto_df.IsTrading)

array([ True])

In [100]:
# Keep all the cryptocurrencies that have a working algorithm.
np.unique(crypto_df.Algorithm)  # There are no columns without working algorithms

array(['1GB AES Pattern Search', '536', 'Argon2', 'Argon2d', 'BLAKE256',
       'Blake', 'Blake2S', 'Blake2b', 'C11', 'Cloverhash', 'Counterparty',
       'CryptoNight', 'CryptoNight Heavy', 'CryptoNight Heavy X',
       'CryptoNight-Lite', 'CryptoNight-V7', 'CryptoNight-lite',
       'Cryptonight-GPU', 'Curve25519', 'DPoS', 'Dagger',
       'Dagger-Hashimoto', 'ECC 256K1', 'Equihash', 'Equihash+Scrypt',
       'Equihash1927', 'Ethash', 'Exosis', 'Green Protocol', 'Groestl',
       'HMQ1725', 'HybridScryptHash256', 'IMesh', 'Jump Consistent Hash',
       'Keccak', 'Leased POS', 'Lyra2RE', 'Lyra2REv2', 'Lyra2Z', 'M7 POW',
       'Momentum', 'Multiple', 'NIST5', 'NeoScrypt', 'Ouroboros',
       'PHI1612', 'POS 2.0', 'POS 3.0', 'PoS', 'Progressive-n',
       'Proof-of-Authority', 'Proof-of-BibleHash', 'QUAIT', 'QuBit',
       'Quark', 'QuarkTX', 'Rainforest', 'SHA-256', 'SHA-256 + Hive',
       'SHA-256D', 'SHA-512', 'SHA3', 'SHA3-256', 'Scrypt', 'Scrypt-n',
       'Semux BFT consensus', 

In [101]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=['IsTrading'])

In [102]:
len(crypto_df)

1144

In [103]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
len(crypto_df)

685

In [104]:
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [105]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df.apply(
    lambda row: row["TotalCoinsMined"] != 0, axis=1)]
len(crypto_df)


533

In [108]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = crypto_df.iloc[:, 0:1]
crypto_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [109]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns="CoinName", inplace=True)

In [111]:
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [143]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [154]:
# Using PCA to reduce dimension to three principal components.
X_pca = PCA(n_components=3).fit_transform(X)


In [164]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=X_pca, columns=["PC 1", "PC 2", "PC 3"])
pcs_df = pcs_df.set_index(crypto_df.index)
pcs_df.head(10)


,PC 1,PC 2,PC 3
42,-0.334815,0.974565,-0.481614
404,-0.318125,0.974666,-0.482076
1337,2.296892,1.659245,-0.594772
BTC,-0.145035,-1.277566,0.182639
ETH,-0.144881,-1.997113,0.321603
LTC,-0.161816,-1.171525,0.012072
DASH,-0.387923,1.297584,-0.460690
XMR,-0.156348,-2.175064,0.400640
ETC,-0.143321,-1.997218,0.321575
ZEC,-0.153048,-2.103990,0.426244


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [160]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks = k)

c:\Users\justi\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [161]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 0 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 0 0 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 3 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

,PC 1,PC 2,PC 3
42,-0.334815,0.974565,-0.481614
404,-0.318125,0.974666,-0.482076
1337,2.296892,1.659245,-0.594772
BTC,-0.145035,-1.277566,0.182639
ETH,-0.144881,-1.997113,0.321603


In [172]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_names.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334815,0.974565,-0.481614,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.318125,0.974666,-0.482076,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.296892,1.659245,-0.594772,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145035,-1.277566,0.182639,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.144881,-1.997113,0.321603,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.161816,-1.171525,0.012072,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.387923,1.297584,-0.460690,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.156348,-2.175064,0.400640,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.143321,-1.997218,0.321575,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.153048,-2.103990,0.426244,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [189]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df
    , x="PC 1"
    , y="PC 2"
    , z="PC 3"
    , color='Class'
    , symbol="Class"
    , width=1200
    , height=1000
    , hover_name="CoinName"
    , hover_data=[clustered_df.Algorithm]
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
